In [4]:
pip install transformers

     |████████████████████████████████| 2.5MB 8.5MB/s 
     |████████████████████████████████| 901kB 34.1MB/s 
     |████████████████████████████████| 3.3MB 37.0MB/s 


In [5]:
###### Imports ######
# sentiment analysis tools
import transformers
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.cluster import KMeans
from collections import defaultdict

In [ ]:
def get_word_idx(sent: str, word: str):
    return sent.split(" ").index(word)


def get_hidden_states(encoded, token_ids_word, model, layers):
  """Push input IDs through model. Stack and sum `layers` (last four by default).
  Select only those subword token outputs that belong to our word of interest
  and average them."""
  with torch.no_grad():
    output = model(**encoded)

  # Get all hidden states
  states = output.hidden_states
  # Stack and sum all requested layers
  output = torch.stack([states[i] for i in layers]).sum(0).squeeze()
  # Only select the tokens that constitute the requested word
  word_tokens_output = output[token_ids_word]

  return word_tokens_output.mean(dim=0)


def get_word_vector(sent, idx, tokenizer, model, layers):
  """Get a word vector by first tokenizing the input sentence, getting all token idxs
    that make up the word of interest, and then `get_hidden_states`."""
  encoded = tokenizer.encode_plus(sent, return_tensors="pt")
  # get all token idxs that belong to the word of interest
  token_ids_word = np.where(np.array(encoded.word_ids()) == idx)

  return get_hidden_states(encoded, token_ids_word, model, layers)


def main(layers=None):
  # Use last four layers by default
  layers = [-4, -3, -2, -1] if layers is None else layers
  tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
  model = AutoModel.from_pretrained("bert-base-cased", output_hidden_states=True)

  sent = "I like cookies ." 
  idx = get_word_idx(sent, "cookies")

  word_embedding = get_word_vector(sent, idx, tokenizer, model, layers)

  return word_embedding 


if __name__ == '__main__':
    main()

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
from glob import glob
dir = '/content'
files = glob(dir + '/*.csv')

df = pd.concat([pd.read_csv(file) for file in files])

In [8]:
df.drop(['index','author','publisher','text'], axis = 1, inplace = True)

df['date'] = df['date'].apply(lambda x: int(x[2:6]) if x[2:6] != 'Date' else 0)

df = df[(df['date'] <= 1660) & (df['date'] >= 1500)]

print(df.shape)
df.head()

(4672, 2)


,title,date
0,the passoinate sic morris,1593
1,a sermon preached at paules-crosse the second ...,1623
2,the copy of a letter sent into england by a ge...,1590
3,gods vvarning to his people of england by the ...,1607
4,a letter sent from a banished minister of iesu...,1554


In [9]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [10]:
encoded = tokenizer.encode_plus(df['title'].tolist()[0], return_tensors = 'pt')

In [ ]:
layers = [-4,-3,-2,-1]

In [ ]:
model = AutoModel.from_pretrained("bert-base-uncased", output_hidden_states=True)

with torch.no_grad():
  output = model(**encoded)

states = output.hidden_states

output = torch.stack([states[i] for i in layers]).sum(0).squeeze()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
output

tensor([[-4.2848, -0.2875, -0.7724,  ..., -1.1247,  1.3356,  2.3251],
        [-0.3014, -0.9044, -0.8353,  ..., -0.0963,  0.8371, -1.0966],
        [-6.8046, -0.7757,  2.3473,  ..., -1.7642, -2.0331,  4.3056],
        ...,
        [-0.6115,  1.4700, -1.9458,  ..., -0.3896,  0.3595, -0.0752],
        [-3.1347,  3.3033, -1.7797,  ..., -0.7044,  1.8633, -3.0247],
        [ 0.7219, -0.0076, -0.6725,  ...,  0.1589, -0.6986,  0.0650]])

In [15]:
def get_embeddings(titles: list, layers: list = None):
  encoded = tokenizer.encode_plus(titles, return_tensors = 'pt')
  layers = [-4,-3,-2,-1] if not layers else layers
  model = AutoModel.from_pretrained("bert-base-uncased", output_hidden_states=True)

  with torch.no_grad():
    output = model(**encoded)

  states = output.hidden_states

  output = torch.stack([states[i] for i in layers]).sum(0).squeeze()

  return output

In [16]:
df['embeddings'] = df['title'].apply(get_embeddings)

Streaming output truncated to the last 5000 lines.
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another t

In [18]:
df.head()

,title,date,embeddings
0,the passoinate sic morris,1593,"[[tensor(-3.6885), tensor(0.6128), tensor(-2.5..."
1,a sermon preached at paules-crosse the second ...,1623,"[[tensor(-2.7243), tensor(-2.0041), tensor(-1...."
2,the copy of a letter sent into england by a ge...,1590,"[[tensor(-2.0613), tensor(-0.8223), tensor(-0...."
3,gods vvarning to his people of england by the ...,1607,"[[tensor(-0.7812), tensor(0.0690), tensor(0.37..."
4,a letter sent from a banished minister of iesu...,1554,"[[tensor(-2.6027), tensor(-0.9568), tensor(-2...."
